# Setup and Import Library

In [1]:
!pip install -qU transformers perf
!pip install langchain langchain-core datasets sentence-transformers rank_bm25 bitsandbytes
!pip install "qdrant-client" "colpali_engine" "datasets" "huggingface_hub[hf_transfer]" "pdf2image" "bitsandbytes"
# !sudo apt-get install poppler-utils
!pip install ragas
!pip install langchain-google-genai
!pip install -qU qwen-vl-utils  # Required for Qwen2.5-VL model
!pip install -q protobuf

ERROR: Could not find a version that satisfies the requirement perf (from versions: none)
ERROR: No matching distribution found for perf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 838.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 66.0 MB/s eta 0:0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 29.9 MB/s eta 0:00:00


In [2]:

import gdown
import os
import json

In [3]:
import os
import json
import shutil
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
from colpali_engine.models import ColPali, ColPaliProcessor
from datasets import load_dataset
from qdrant_client import QdrantClient, models
import torch
from tqdm import tqdm
import uuid
import numpy as np
import random
import os
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from transformers import BitsAndBytesConfig
from PIL import Image

In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoModelForImageTextToText, BitsAndBytesConfig, AutoProcessor
import torch

In [6]:
from datasets import Dataset
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, LLMContextPrecisionWithoutReference
from ragas import evaluate, RunConfig

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-550044629.py:2: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, LLMContextPrecisionWithoutReference
/tmp/ipython-input-550044629.py:2: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Examp

In [7]:
# Load quantization 4bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load LMM Model

In [ ]:
from huggingface_hub import login
login(token = "YOUR_HUGGINGFACEHUB_TOKEN_HERE")

In [9]:
# model_id, output_dir = "Qwen/Qwen2.5-VL-3B-Instruct", "Qwen2.5-VL-3B-Instruct"
model_id, output_dir = "meta-llama/Llama-3.2-3B-Instruct", "Llama-3.2-3B-Instruct"

In [10]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel


base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    torch_dtype=torch.float16,                          # Change to bfloat16 if GPU has support
    use_cache=True,                               # Whether to cache attention outputs to speed up inference
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
        bnb_4bit_compute_dtype=torch.float16,     # Data type used for internal computations in quantization
        bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
        bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    )
)


tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [11]:
# import torch
# from transformers import Qwen2_5_VLForConditionalGeneration, BitsAndBytesConfig, AutoTokenizer
# from peft import PeftModel
# # Update to use your fine-tuned Qwen2.5-VL model
# adapter_model = f"JellyFush/{output_dir}" # Replace with your HF username or organization

# # Use AutoModelForVision2Seq instead of AutoModelForCausalLM
# base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     model_id,
#     attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
#     torch_dtype=torch.bfloat16,                          # Change to bfloat16 if GPU has support
#     use_cache=True,                               # Whether to cache attention outputs to speed up inference
#     quantization_config=BitsAndBytesConfig(
#         load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
#         bnb_4bit_compute_dtype=torch.bfloat16,     # Data type used for internal computations in quantization
#         bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
#         bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
#     )
# )
# # model_name = model_id  # Uncomment to use base model instead
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# processor = AutoProcessor.from_pretrained(model_id)

In [12]:
from peft import PeftModel
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, BitsAndBytesConfig, AutoTokenizer

# adapter_sft = "JellyFush/Llama-3.2-3B-Instruct"
adapter_dpo = "JellyFush/Llama-3.2-3B-Instruct-DPO"
# sft_model = PeftModel.from_pretrained(base_model, adapter_sft, assign=True)
dpo_model = PeftModel.from_pretrained(base_model, adapter_dpo, subfolder="policy", assign=True)
model = dpo_model # or base_model
model.eval()

adapter_config.json: 0.00B [00:00, ?B/s]

policy/adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
                (

# Load Gemini API

In [13]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE"  # From https//ai.google.dev/

In [14]:
config = {
    "model": "gemma-3-27b-it",  # or other model IDs
    "temperature": 0.4,
    "max_tokens": None,
    "top_p": 0.8
}

In [15]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Choose the appropriate import based on your API:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize with Google AI Studio
evaluator_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_tokens=config["max_tokens"],
    top_p=config["top_p"],
))

judge_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_tokens=config["max_tokens"],
    top_p=config["top_p"],
))

/tmp/ipython-input-3987192832.py:8: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  evaluator_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(
/tmp/ipython-input-3987192832.py:15: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  judge_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(


In [16]:
# Google AI Studio Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

evaluator_embeddings = LangchainEmbeddingsWrapper(GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Google's text embedding model
    task_type="retrieval_document"  # Optional: specify the task type
))

/tmp/ipython-input-1044670269.py:4: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  evaluator_embeddings = LangchainEmbeddingsWrapper(GoogleGenerativeAIEmbeddings(


# Demo

In [17]:
# Load questions and ground-truth
import json

def load_qa_data(json_file_path):
    questions = []
    ground_truths = []
    reasonings = []
    images = []

    try:
      with open(json_file_path, 'r', encoding='utf-8') as f:
          data = json.load(f)

          if not isinstance(data, list):
              print(f"Error: Expected JSON file to contain a list of objects.")
              return None, None

          for item in data:
              if 'question_clinical_vignette' in item and 'answer' in item and 'analytic_chain_of_thought' in item:
                  questions.append(item['question_clinical_vignette'])
                  ground_truths.append(item['answer'])
                  reasonings.append(item['analytic_chain_of_thought'])
                  if 'image' in item:
                      images.append(item['image'])

              else:
                  print(f"Warning: Skipping item, missing 'prompt' or 'diagnosis': {item}")

    except FileNotFoundError:
      print(f"Error: JSON file not found at {json_file_path}")
      return None, None
    except json.JSONDecodeError:
      print(f"Error: Could not decode JSON from {json_file_path}")
      return None, None

    return questions, ground_truths,reasonings, images

In [18]:
import os
import gdown

# https://drive.google.com/file/d/1dpUMPdTu9o5XmU9Pv_Vmy8Udtk9Htkz3/view?usp=sharing
# Google Drive file ID from the sharing link, then change the ID
file_id = "1dpUMPdTu9o5XmU9Pv_Vmy8Udtk9Htkz3"
# URL for downloading
url = f"https://drive.google.com/uc?id={file_id}"

output_path = "diagnosis.json"

# Download the file
gdown.download(url, output_path, quiet=False)

print(f"Downloaded to {output_path}")


Downloading...
From: https://drive.google.com/uc?id=1dpUMPdTu9o5XmU9Pv_Vmy8Udtk9Htkz3
To: /content/diagnosis.json
100%|██████████| 187k/187k [00:00<00:00, 34.0MB/s]

Downloaded to diagnosis.json


# TEST

In [19]:
json_path = "/content/diagnosis.json"
questions, ground_truths, reasonings, images = load_qa_data(json_path)

In [20]:
len(questions)

93

In [21]:
i = 18
questions = [questions[i]]
ground_truths = [ground_truths[i]]
reasonings = [reasonings[i]]

In [22]:
questions, ground_truths, reasonings

(['A 43-year-old male presents to a travel clinic with a 3-day history of recurrent high fever, headache, and diarrhea. He returned one week ago from a 5-week trip to Mozambique and Chile. He did not take malaria chemoprophylaxis. Social history is significant for swimming in a freshwater lake near Maputo. On physical examination, he is febrile (39.2°C) but hemodynamic parameters are stable. There is no rash, lymphadenopathy, or hepatosplenomegaly. Laboratory evaluation reveals a normal total white blood cell count but significant eosinophilia (2.1 x 10^9/L). Liver transaminases (ALT 179 U/L, AST 106 U/L) and CRP are elevated. A chest radiograph demonstrates multiple small, nodular lesions in the periphery of the lower lung zones bilaterally. Thick and thin smears for malaria are negative. What is the most likely diagnosis?'],
 ['Acute Schistosomiasis (Katayama Syndrome)'],
 ["1. Epidemiology/Risk: The patient has a specific high-risk exposure: swimming in freshwater in a Schistosoma-e

In [23]:
if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [24]:
import torch
from langchain_core.prompts import PromptTemplate

def get_answer_from_question_context(questions, images=None, model=None, processor=None):
    """
    Generate medical diagnoses from questions + optional images.
    """

    answers = []

    # Prompt
    prompt_template = """Question: {question}
Answer:"""
    prompt = PromptTemplate(template=prompt_template, input_variables=["question"])

    # System role
    system_prompt = """Role:
You are an expert Medical Diagnostician and Board Examiner.

Task:
Read the provided Clinical Vignette. Determine the most likely diagnosis.
You must first generate a structured clinical reasoning process inside <think> tags, followed by the final diagnosis as plain text.

Output Format:
<think>
[Explanation]
</think>

[Final Diagnosis Name]
"""

    for i, question in enumerate(questions):

        image = images[i] if images and i < len(images) else None
        formatted_prompt = prompt.format(question=question)

        # Build content
        content = [{"type": "text", "text": formatted_prompt}]
        if image:
            content.insert(0, {"type": "image", "image": image})

        # Chat messages
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content},
        ]

        # Create prompt text
        text = processor.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # Convert text to model input
        inputs = processor(
            text=[text],            # IMPORTANT: wrap in list
            return_tensors="pt"
        ).to(model.device)

        # Generate
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=4096,
            )

        # Slice output only AFTER prompt tokens
        output_ids = output[0][len(inputs.input_ids[0]):]

        # Decode
        generated_text = processor.decode(
            output_ids,
            skip_special_tokens=True
        ).strip()

        print(generated_text)

        # ---- Extract reasoning + diagnosis ----
        result = {}

        if "<think>" in generated_text and "</think>" in generated_text:
            t_start = generated_text.find("<think>") + len("<think>")
            t_end = generated_text.find("</think>")
            reasoning = generated_text[t_start:t_end].strip()
            result["reasoning"] = reasoning

            # diagnosis = text after </think>
            remaining = generated_text[t_end + len("</think>"):].strip()
            result["diagnosis"] = remaining

        answers.append(result)

        torch.cuda.empty_cache()

    return answers


In [25]:
answers = get_answer_from_question_context(
    questions=questions,
    images=images,
    model=model,
    processor=tokenizer
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<think>
1. Epidemiology/Risk: The patient recently traveled to Mozambique, a region endemic for schistosomiasis, and was exposed to a freshwater lake, increasing the risk of Schistosoma infection. 2. Clinical Anchor: The patient presents with a classic triad of fever, eosinophilia, and gastrointestinal symptoms, with the latter being more prominent in the early stages. The absence of a rash, lymphadenopathy, or hepatosplenomegaly reduces the likelihood of other causes. 3. Differential Exclusion: Malaria (negative thick and thin smears), bacterial infections (normal WBC), and viral infections (normal WBC and lack of systemic symptoms) are excluded. 4. Diagnosis: The most likely diagnosis is Schistosomiasis (Schistosoma mansoni). The presence of the Schistosoma egg shell granuloma in the liver is confirmed by the histopathological examination of the liver biopsy, which is the hallmark of the disease. The diagnosis is further supported by the patient's exposure history and the clinical pr

In [26]:
answers

[{'reasoning': "1. Epidemiology/Risk: The patient recently traveled to Mozambique, a region endemic for schistosomiasis, and was exposed to a freshwater lake, increasing the risk of Schistosoma infection. 2. Clinical Anchor: The patient presents with a classic triad of fever, eosinophilia, and gastrointestinal symptoms, with the latter being more prominent in the early stages. The absence of a rash, lymphadenopathy, or hepatosplenomegaly reduces the likelihood of other causes. 3. Differential Exclusion: Malaria (negative thick and thin smears), bacterial infections (normal WBC), and viral infections (normal WBC and lack of systemic symptoms) are excluded. 4. Diagnosis: The most likely diagnosis is Schistosomiasis (Schistosoma mansoni). The presence of the Schistosoma egg shell granuloma in the liver is confirmed by the histopathological examination of the liver biopsy, which is the hallmark of the disease. The diagnosis is further supported by the patient's exposure history and the cli

In [27]:
for answer in answers:
    print(answer.get('diagnosis'))

Schistosomiasis (Schistosoma mansoni)


In [28]:
ground_truths

['Acute Schistosomiasis (Katayama Syndrome)']

In [29]:
import json
import re
from ragas import EvaluationDataset
from ragas.metrics import FactualCorrectness, LLMContextRecall

/tmp/ipython-input-2593537447.py:4: DeprecationWarning: Importing FactualCorrectness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import FactualCorrectness
  from ragas.metrics import FactualCorrectness, LLMContextRecall
/tmp/ipython-input-2593537447.py:4: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import FactualCorrectness, LLMContextRecall


In [30]:
if not (len(questions) == len(answers) == len(ground_truths)):
    print("Error: Lengths of questions, answers, and ground_truths do not match.")
else:
    dataset = []
    for question, ground_truth, answer in zip(questions, ground_truths, answers):
        response_text = answer.get('diagnosis') if answer.get('diagnosis') else "None"

        dataset.append({
            "user_input": question,
            "response": response_text,
            "reference": ground_truth
        })

    # Create evaluation dataset
    evaluation_dataset = EvaluationDataset.from_list(dataset)

    # # Save dataset

    # with open(save_path, "w") as f:
    #     json.dump(dataset, f, indent=2, ensure_ascii=False)

    # print(f"Dataset saved successfully at {save_path}")
    # print(f"Total samples: {len(dataset)}")

In [31]:
dataset

[{'user_input': 'A 43-year-old male presents to a travel clinic with a 3-day history of recurrent high fever, headache, and diarrhea. He returned one week ago from a 5-week trip to Mozambique and Chile. He did not take malaria chemoprophylaxis. Social history is significant for swimming in a freshwater lake near Maputo. On physical examination, he is febrile (39.2°C) but hemodynamic parameters are stable. There is no rash, lymphadenopathy, or hepatosplenomegaly. Laboratory evaluation reveals a normal total white blood cell count but significant eosinophilia (2.1 x 10^9/L). Liver transaminases (ALT 179 U/L, AST 106 U/L) and CRP are elevated. A chest radiograph demonstrates multiple small, nodular lesions in the periphery of the lower lung zones bilaterally. Thick and thin smears for malaria are negative. What is the most likely diagnosis?',
  'response': 'Schistosomiasis (Schistosoma mansoni)',
  'reference': 'Acute Schistosomiasis (Katayama Syndrome)'}]

In [32]:
# import json
# import re
# from ragas import EvaluationDataset
# from ragas.metrics import FactualCorrectness, LLMContextRecall

# with open('/kaggle/working/eval_dataset_disease.json', 'r') as file:
#     dataset = json.load(file)

# evaluation_dataset = EvaluationDataset.from_list(dataset)

In [33]:
# Metrics: Factual Correctness and LLM Context Recall
# Factual Correctness: Evaluates the factual accuracy of the model's answers against the reference.
# LLM Context Recall: Measures how well the model's response covers the information in the reference.

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[FactualCorrectness()],
    llm=evaluator_llm,
    run_config=RunConfig(
        timeout=300,
        max_workers=1,  # Process one at a time
        max_wait=60,     # Wait up to 60s between retries
    )
)
result

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

{'factual_correctness(mode=f1)': 0.0000}

In [34]:
from ragas.dataset_schema import EvaluationDataset
import json

evaluation_rows = []

for i, (answer, reasoning) in enumerate(zip(answers, reasonings)):

    if isinstance(answer, dict) and "reasoning" in answer:
        answer_reasoning = answer["reasoning"]
    else:
        print(f"Skipping index {i}: invalid answer format → {answer}")
        continue

    evaluation_rows.append({
        "user_input": questions[i] if questions else f"case_{i}",
        "response": answer_reasoning,
        "reference": reasoning
    })

In [35]:
SOFT_JUDGE_PROMPT = """You are evaluating reasoning recall.

Ground-truth reason:
"{model_reason}"

Model-generated reasoning:
"{answer_reasoning}"

Task:
Estimate how much of the ground-truth reason is covered by the model reasoning,
even if paraphrased or implicit.

Scoring:
1.0 = fully covered
0.5 = partially covered
0.0 = not covered

Answer with ONLY one of: 1.0, 0.5, 0.0
"""

In [36]:
def split_reasons(text):
    # Works for numbered or paragraph-style reasoning
    parts = re.split(r"\n|\d+\.\s*", text)
    return [p.strip() for p in parts if len(p.strip()) > 20]
def soft_reasoning_recall_case(answer_reasoning, model_reasoning, llm):
    """
    model_reasoning  = ground-truth reasoning
    answer_reasoning = model-generated reasoning
    """
    gold_reasons = split_reasons(model_reasoning)
    if not gold_reasons:
        return 0.0

    score_sum = 0.0

    for reason in gold_reasons:
        prompt = SOFT_JUDGE_PROMPT.format(
            model_reason=reason,
            answer_reasoning=answer_reasoning
        )

        response = llm.langchain_llm.invoke(prompt)

        try:
            score = float(response.content.strip())
        except Exception:
            score = 0.0

        score_sum += score

    return score_sum / len(gold_reasons)

In [37]:
evaluation_rows

[{'user_input': 'A 43-year-old male presents to a travel clinic with a 3-day history of recurrent high fever, headache, and diarrhea. He returned one week ago from a 5-week trip to Mozambique and Chile. He did not take malaria chemoprophylaxis. Social history is significant for swimming in a freshwater lake near Maputo. On physical examination, he is febrile (39.2°C) but hemodynamic parameters are stable. There is no rash, lymphadenopathy, or hepatosplenomegaly. Laboratory evaluation reveals a normal total white blood cell count but significant eosinophilia (2.1 x 10^9/L). Liver transaminases (ALT 179 U/L, AST 106 U/L) and CRP are elevated. A chest radiograph demonstrates multiple small, nodular lesions in the periphery of the lower lung zones bilaterally. Thick and thin smears for malaria are negative. What is the most likely diagnosis?',
  'response': "1. Epidemiology/Risk: The patient recently traveled to Mozambique, a region endemic for schistosomiasis, and was exposed to a freshwa

In [38]:
scores = []

for row in evaluation_rows:
    c_i = soft_reasoning_recall_case(
        answer_reasoning=row["response"],   # generated
        model_reasoning=row["reference"],   # ground truth
        llm=judge_llm
    )
    scores.append(c_i)

RR = sum(scores) / len(scores)
print("Soft Reasoning Recall:", RR)

Soft Reasoning Recall: 0.625
